<a href="https://colab.research.google.com/github/Xujjjjun2002/AI/blob/main/Inception_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary
import numpy as np

In [2]:
class BasicConv(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride=1,padding=0):
        super(BasicConv,self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride,padding=padding)
        self.relu = nn.ReLU(inplace=True)
    def forward(self,x):
        x = self.conv(x)
        x = self.relu(x)
        return xInception_v1

In [3]:
class InceptionBlock(nn.Module):
    def __init__(self,in_channels,ch1x1,ch3x3reduce,ch3x3,ch5x5reduce,ch5x5,chpool):
        super(InceptionBlock,self).__init__()
        self.branch_1 = BasicConv(in_channels=in_channels,out_channels=ch1x1,kernel_size=1)
        self.branch_2 = nn.Sequential(
            BasicConv(in_channels=in_channels,out_channels=ch3x3reduce,kernel_size=1),
            BasicConv(in_channels=ch3x3reduce,out_channels=ch3x3,kernel_size=3,padding=1)
        )
        self.branch_3 = nn.Sequential(
            BasicConv(in_channels=in_channels,out_channels=ch5x5reduce,kernel_size=1),
            BasicConv(in_channels=ch5x5reduce,out_channels=ch5x5,kernel_size=5,padding=2)
        )
        self.branch_4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding = 1),
            BasicConv(in_channels=in_channels,out_channels=chpool,kernel_size=1)
        )
    def forward(self,x):
        x_1 = self.branch_1(x)
        x_2 = self.branch_2(x)
        x_3 = self.branch_3(x)
        x_4 = self.branch_4(x)
        x = torch.cat([x_1,x_2,x_3,x_4],dim=1)
        return x

In [4]:
class Inception_v1(nn.Module):
    def __init__(self,num_classes):
        super(Inception_v1,self).__init__()
        self.BasicConv_1 = BasicConv(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
        self.max_pool_1 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode=True)
        self.conv_1x1 = BasicConv(in_channels=64,out_channels=64,kernel_size=1)
        self.conv_3x3 = BasicConv(in_channels=64,out_channels=192,kernel_size=3,padding=1)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode=True)
        self.InceptionBlock_3a = InceptionBlock(192,64,96,128,16,32,32)
        self.InceptionBlock_3b = InceptionBlock(256,128,128,192,32,96,64)
        self.max_pool_3 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.InceptionBlock_4a = InceptionBlock(480,192,96,208,16,48,64)
        self.InceptionBlock_4b = InceptionBlock(512,160,112,224,24,64,64)
        self.InceptionBlock_4c = InceptionBlock(512,128,128,256,24,64,64)
        self.InceptionBlock_4d = InceptionBlock(512,112,144,288,32,64,64)
        self.InceptionBlock_4e = InceptionBlock(528,256,160,320,32,128,128)
        self.max_pool_4 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode=True)
        self.InceptionBlock_5a = InceptionBlock(832,256,160,320,32,128,128)
        self.InceptionBlock_5b = InceptionBlock(832,384,192,384,48,128,128)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(in_features=1024,out_features=num_classes)
    def forward(self, x):
        x = self.BasicConv_1(x)
        x = self.max_pool_1(x)
        x = self.conv_1x1(x)
        x = self.conv_3x3(x)
        x = self.max_pool_2(x)
        x = self.InceptionBlock_3a(x)
        x = self.InceptionBlock_3b(x)
        x = self.max_pool_3(x)
        x = self.InceptionBlock_4a(x)
        x = self.InceptionBlock_4b(x)
        x = self.InceptionBlock_4c(x)
        x = self.InceptionBlock_4d(x)
        x = self.InceptionBlock_4e(x)
        x = self.max_pool_4(x)
        x = self.InceptionBlock_5a(x)
        x = self.InceptionBlock_5b(x)
        x = self.avg_pool(x)
        x = self.flatten(x)
        x = torch.dropout(x,0.4,train=True)
        x = self.fc(x)
        x = torch.softmax(x,dim=1)
        return x

In [5]:
if __name__ == '__main__':
    input = torch.ones([10,3,224,224])

    model = Inception_v1(num_classes=20)

    result = model(input)

    print("resule.shape = ",result.shape)

    summary(model.to("cuda"),(3,224,224))

resule.shape =  torch.Size([10, 20])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         BasicConv-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,160
              ReLU-6           [-1, 64, 56, 56]               0
         BasicConv-7           [-1, 64, 56, 56]               0
            Conv2d-8          [-1, 192, 56, 56]         110,784
              ReLU-9          [-1, 192, 56, 56]               0
        BasicConv-10          [-1, 192, 56, 56]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,352
             ReLU-13           [-1, 64, 28, 28]               0
  